In [1]:
import os
%pwd
os.chdir('../')
%pwd

'C:\\Users\\Franklin NDJATHE\\Documents\\projects\\End-to-End-ML-Project-with-MLFlow'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [3]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [4]:
from typing import ClassVar
from box import ConfigBox

@dataclass
class ConfigurationManager:
    config_filepath: Path = CONFIG_FILE_PATH
    params_filepath: Path = PARAMS_FILE_PATH
    schema_filepath: Path = SCHEMA_FILE_PATH
    
    config: ClassVar[ConfigBox] = read_yaml(config_filepath)
    params: ClassVar[ConfigBox] = read_yaml(params_filepath)
    schema: ClassVar[ConfigBox] = read_yaml(schema_filepath)
    
    def __post_init__(self):
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        
        return data_transformation_config

[2023-07-29 16:18:01,897: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-29 16:18:01,899: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-29 16:18:01,901: INFO: common: yaml file: schema.yaml loaded successfully]


In [5]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

[2023-07-29 16:18:33,093: INFO: utils: NumExpr defaulting to 8 threads.]


In [6]:
@dataclass
class DataTransformation:
    config: DataTransformationConfig
    
    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)
        
        # Split the data into training and test sets. (0.75, 0.25) split
        train, test = train_test_split(data)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        
        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)
        
        print(train.shape)
        print(test.shape)

In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2023-07-29 16:29:18,344: INFO: common: created directory at: artifacts]
[2023-07-29 16:29:18,346: INFO: common: created directory at: artifacts/data_transformation]
[2023-07-29 16:29:18,366: INFO: 119938449: Splited data into training and test sets]
[2023-07-29 16:29:18,367: INFO: 119938449: (1199, 12)]
[2023-07-29 16:29:18,368: INFO: 119938449: (400, 12)]
(1199, 12)
(400, 12)
